Pulled Dataset from Kaggle.com <br>
Microsoft Excel - Opened .csv. Looked through columns to verify data types, presence of empty/incomplete data, dropped some irrelevant columns <br>
Original - 20633x61 <br>
After Excel - 20633x56 <br>
Saved as WORKING .csv + .xls

In [17]:
#Import working file so I don't ruin the original and libraries
import re
import copy
import numpy as np
import pandas as pd

df = pd.read_csv('./kickstarter_data_with_featuresWorking.csv')
#Check column naming out
df.columns.unique()

C:\Users\isits\AppData\Local\Temp\ipykernel_16092\233875786.py:7: DtypeWarning: Columns (30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./kickstarter_data_with_featuresWorking.csv')


Index(['index', 'Unnamed: 0', 'id', 'photo', 'name', 'blurb', 'goal',
       'pledged', 'state', 'slug', 'disable_communication', 'country',
       'currency', 'currency_symbol', 'currency_trailing_code', 'deadline',
       'state_changed_at', 'created_at', 'launched_at', 'staff_pick',
       'backers_count', 'static_usd_rate', 'usd_pledged', 'creator',
       'location', 'category', 'profile', 'spotlight', 'urls', 'source_url',
       'friends', 'is_starred', 'is_backing', 'permissions', 'name_len',
       'name_len_clean', 'blurb_len', 'blurb_len_clean', 'deadline_weekday',
       'state_changed_at_weekday', 'created_at_weekday', 'launched_at_weekday',
       'deadline_month', 'deadline_day', 'deadline_yr', 'deadline_hr',
       'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_yr',
       'state_changed_at_hr', 'created_at_month', 'created_at_day',
       'created_at_yr', 'created_at_hr', 'launched_at_month',
       'launched_at_day', 'launched_at_yr', 'launched_at

In [18]:
# Read columns and pick important with df.columns()
final_columns = ['name', 'goal', 'pledged', 'state','source_url', 'country', 'currency', 'staff_pick','backers_count','usd_pledged','category', 'spotlight','deadline_month','deadline_yr','created_at_month','created_at_yr','launched_at_month','launched_at_yr','create_to_launch','launch_to_deadline']
final= df[final_columns]
#final count is 20632x19

In [19]:
# #Checking for Null values
print(final.isnull().any())         #By Column
print(final.isnull().sum().sum())   #By total count

name                  False
goal                  False
pledged               False
state                 False
source_url            False
country               False
currency              False
staff_pick            False
backers_count         False
usd_pledged           False
category               True
spotlight             False
deadline_month        False
deadline_yr           False
created_at_month      False
created_at_yr         False
launched_at_month     False
launched_at_yr        False
create_to_launch      False
launch_to_deadline    False
dtype: bool
1889


In [20]:
#Checking Category column for types and naming
final.category.unique()

array(['Academic', 'Places', nan, 'Blues', 'Restaurants', 'Webseries',
       'Thrillers', 'Shorts', 'Web', 'Apps', 'Gadgets', 'Hardware',
       'Festivals', 'Plays', 'Musical', 'Flight', 'Spaces', 'Immersive',
       'Experimental', 'Comedy', 'Wearables', 'Sound', 'Software',
       'Robots', 'Makerspaces'], dtype=object)

In [21]:
#Check state for types to drop
final.state.unique()

array(['failed', 'canceled', 'successful', 'live', 'suspended'],
      dtype=object)

In [22]:

# Drop row if state is live as they are not a finished campaign. Cancelled, because we can't really quantify why they did it. Suspended because they done messed up A-A-Ron
final = final[df.state != 'live']
final = final[final.state != 'canceled']
final = final[final.state != 'suspended']
#Checking state column after dropping live and cancelled
final.state.unique()

array(['failed', 'successful'], dtype=object)

In [23]:
#Make a deep copy before adjusting any values
final2 = final.copy(deep=True)

In [24]:
#Check out create + lauch for adjustments
print(final2.create_to_launch[0])
print(final2.launch_to_deadline[0])

17 days 14:51:39.000000000
36 days 20:47:24.000000000


In [25]:
# Drop trailing zeroes in created and launched time columns using casting and map/lambda
final2.create_to_launch = final2.create_to_launch.astype(str).map(lambda x: x[0:2])
final2.launch_to_deadline = final2.launch_to_deadline.astype(str).map(lambda x: x[0:2])
print(final2.create_to_launch[0])
print(final2.launch_to_deadline[0])

17
36


In [26]:
#Need to replace all NAN values with category info from source_url column
#Replace all instances of nan with a corresponding row from other column called
final2.category = final2.category.combine_first(final2.source_url).astype(str)
final2.category.unique()

array(['Academic', 'Places',
       'https://www.kickstarter.com/discover/categories/music/electronic%20music?ref=category_modal&sort=magic',
       'https://www.kickstarter.com/discover/categories/music/country%20&%20folk?ref=category_modal&sort=magic',
       'https://www.kickstarter.com/discover/categories/music/classical%20music?ref=category_modal&sort=magic',
       'Blues',
       'https://www.kickstarter.com/discover/categories/games/video%20games?ref=category_modal&sort=magic',
       'Webseries', 'Thrillers', 'Shorts',
       'https://www.kickstarter.com/discover/categories/film%20&%20video/narrative%20film?ref=category_modal&sort=magic',
       'Web', 'Apps', 'Gadgets', 'Hardware', 'Festivals', 'Plays',
       'Musical', 'Flight',
       'https://www.kickstarter.com/discover/categories/technology/diy%20electronics?ref=category_modal&sort=magic',
       'Spaces', 'Immersive', 'Experimental', 'Comedy', 'Wearables',
       'https://www.kickstarter.com/discover/categories/technol

In [27]:
#Replace all categories source URL with new categories, convert to raw strings and escape ? as %20 and ? are showing up reserved
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/music/electronic%20music\?ref=category_modal&sort=magic', 'Electronic Music', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/music/country%20&%20folk\?ref=category_modal&sort=magic', 'Country & Folk Music', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/music/classical%20music\?ref=category_modal&sort=magic', 'Classical Music', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/games/video%20games\?ref=category_modal&sort=magic', 'Video Games', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/film%20&%20video/narrative%20film\?ref=category_modal&sort=magic', 'Film & Video', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/technology/diy%20electronics\?ref=category_modal&sort=magic', 'DIY Electronics', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/technology/space%20exploration\?ref=category_modal&sort=magic', 'Space Exploration', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/technology/fabrication%20tools\?ref=category_modal&sort=magic', 'Fabrication Tools', regex=True)
final2.category = final2.category.replace(r'https://www.kickstarter.com/discover/categories/technology/camera%20equipment\?ref=category_modal&sort=magic', 'Camera Equipment', regex=True)
final2.category.unique()

array(['Academic', 'Places', 'Electronic Music', 'Country & Folk Music',
       'Classical Music', 'Blues', 'Video Games', 'Webseries',
       'Thrillers', 'Shorts', 'Film & Video', 'Web', 'Apps', 'Gadgets',
       'Hardware', 'Festivals', 'Plays', 'Musical', 'Flight',
       'DIY Electronics', 'Spaces', 'Immersive', 'Experimental', 'Comedy',
       'Wearables', 'Space Exploration', 'Sound', 'Software', 'Robots',
       'Makerspaces', 'Fabrication Tools', 'Camera Equipment'],
      dtype=object)

In [28]:
#Checking for Null values due to paranoia
print(final2.isnull().any())         #By Column
print(final2.isnull().sum().sum())   #By total count

name                  False
goal                  False
pledged               False
state                 False
source_url            False
country               False
currency              False
staff_pick            False
backers_count         False
usd_pledged           False
category              False
spotlight             False
deadline_month        False
deadline_yr           False
created_at_month      False
created_at_yr         False
launched_at_month     False
launched_at_yr        False
create_to_launch      False
launch_to_deadline    False
dtype: bool
0


In [29]:
# Drop the source_url after taking info we need
final_columns2 = ['name', 'goal', 'pledged', 'state', 'country', 'currency', 'staff_pick','backers_count','usd_pledged','category', 'spotlight','deadline_month','deadline_yr','created_at_month','created_at_yr','launched_at_month','launched_at_yr','create_to_launch','launch_to_deadline']
final2 = final2[final_columns2]

In [30]:
#Check for accurate column drop
final2.columns.unique()

Index(['name', 'goal', 'pledged', 'state', 'country', 'currency', 'staff_pick',
       'backers_count', 'usd_pledged', 'category', 'spotlight',
       'deadline_month', 'deadline_yr', 'created_at_month', 'created_at_yr',
       'launched_at_month', 'launched_at_yr', 'create_to_launch',
       'launch_to_deadline'],
      dtype='object')

In [31]:
#Check for category fixed?
final2.category.unique()

array(['Academic', 'Places', 'Electronic Music', 'Country & Folk Music',
       'Classical Music', 'Blues', 'Video Games', 'Webseries',
       'Thrillers', 'Shorts', 'Film & Video', 'Web', 'Apps', 'Gadgets',
       'Hardware', 'Festivals', 'Plays', 'Musical', 'Flight',
       'DIY Electronics', 'Spaces', 'Immersive', 'Experimental', 'Comedy',
       'Wearables', 'Space Exploration', 'Sound', 'Software', 'Robots',
       'Makerspaces', 'Fabrication Tools', 'Camera Equipment'],
      dtype=object)

In [ ]:
final2

In [ ]:
#CELEBRATE 
#Export and get plotting
final2.to_csv('CleanedKickstarterCapstone1.csv')
final2.to_excel('CleanedKickstarterCapstone1.xlsx')

After column drop with pandas - 20632x20 <br>
After dropping live, cancelled, suspended and ['source_url'] <br>
Final to csv/xlsx - 17434x19